# Data Science Assignment 1
Kishan Ved

22110122

In [656]:
import pandas as pd
import numpy as np

df = pd.read_csv('musae_facebook_target.csv');

In [657]:
df

,id,facebook_id,page_name,page_type
0,0,145647315578475,The Voice of China 中国好声音,tvshow
1,1,191483281412,U.S. Consulate General Mumbai,government
2,2,144761358898518,ESET,company
3,3,568700043198473,Consulate General of Switzerland in Montreal,government
4,4,1408935539376139,Mark Bailey MP - Labor for Miller,politician
...,...,...,...,...
22465,22465,1379955382222841,Kurt Wiegel MdL,politician
22466,22466,1651527995097082,dubdub Stories,company
22467,22467,155369444540412,Ministerio del Interior - Paraguay,government
22468,22468,175067819212798,Tottus Perú,company


In [658]:
edges_df = pd.read_csv("musae_facebook_edges.csv")

In [659]:
edges_df

,id_1,id_2
0,0,18427
1,1,21708
2,1,22208
3,1,22171
4,1,6829
...,...,...
170997,20188,20188
170998,22340,22383
170999,22348,22348
171000,5563,5563


### Road map

1. tot - deg[v] = e(s,s)  {deg is a dict containing degrees}

2. Remove v from deg

3. Update deg of vertices that were connected to v

4. Find the density 

5. Store the vertex removed and the density of the subgraph obtained after this

In [660]:
deg = {}
neighbors = {}
self_loops = {}

for _, row in edges_df.iterrows():
    if row['id_1'] == row['id_2']:
        if row['id_1'] in self_loops:
            self_loops[row['id_1']] += 1
        else:
            self_loops[row['id_1']] = 1

    for node in ['id_1', 'id_2']:
        if row[node] in deg:
            deg[row[node]] += 1
        else:
            deg[row[node]] = 1

    # Append to the neighbors list
    if row['id_1'] in neighbors:
        neighbors[row['id_1']].append(row['id_2'])
    else:
        neighbors[row['id_1']] = [row['id_2']]

    if row['id_2'] in neighbors:
        neighbors[row['id_2']].append(row['id_1'])
    else:
        neighbors[row['id_2']] = [row['id_1']]


In [661]:
deg # dictionary of degrees

{0: 1,
 18427: 51,
 1: 34,
 21708: 195,
 22208: 205,
 22171: 330,
 6829: 27,
 16590: 87,
 20135: 140,
 8894: 52,
 15785: 147,
 10281: 13,
 22265: 226,
 7136: 25,
 22405: 41,
 10379: 468,
 13737: 62,
 8533: 36,
 14344: 50,
 2812: 45,
 5755: 68,
 16260: 26,
 15026: 42,
 17370: 193,
 17460: 66,
 8049: 193,
 5307: 122,
 4987: 99,
 18304: 25,
 12305: 67,
 19743: 678,
 20024: 128,
 21729: 659,
 10554: 56,
 11557: 72,
 5228: 64,
 9934: 71,
 2: 12,
 9048: 14,
 6353: 23,
 2629: 2,
 11537: 22,
 13205: 20,
 22304: 20,
 17728: 9,
 19337: 14,
 126: 23,
 17554: 11,
 8495: 17,
 5857: 11,
 3: 10,
 16742: 21,
 293: 59,
 5826: 36,
 3479: 22,
 19753: 32,
 17346: 15,
 10945: 46,
 22338: 46,
 11319: 14,
 9654: 8,
 4: 51,
 13645: 65,
 20876: 32,
 11446: 22,
 16203: 56,
 2830: 20,
 2004: 37,
 20624: 38,
 21280: 47,
 1182: 32,
 21538: 26,
 1443: 71,
 11423: 29,
 187: 142,
 5730: 16,
 2983: 35,
 1489: 22,
 6823: 43,
 17695: 112,
 11102: 158,
 6390: 64,
 17242: 20,
 10018: 32,
 5147: 31,
 6427: 41,
 14628: 70,


In [662]:
self_loops

{7447: 1,
 159: 1,
 250: 1,
 262: 1,
 267: 1,
 504: 1,
 557: 1,
 578: 1,
 614: 1,
 5354: 1,
 1175: 1,
 1472: 1,
 1702: 1,
 1764: 1,
 1808: 1,
 9260: 1,
 1988: 1,
 2304: 1,
 2335: 1,
 2355: 1,
 2413: 1,
 2446: 1,
 2508: 1,
 14985: 1,
 2799: 1,
 2893: 1,
 2933: 1,
 2973: 1,
 3004: 1,
 3023: 1,
 3069: 1,
 3076: 1,
 3786: 1,
 3930: 1,
 4032: 1,
 11896: 1,
 4642: 1,
 4688: 1,
 5114: 1,
 5126: 1,
 5548: 1,
 5656: 1,
 5918: 1,
 6390: 1,
 6469: 1,
 6615: 1,
 6667: 1,
 6783: 1,
 6890: 1,
 6938: 1,
 19856: 1,
 7192: 1,
 7467: 1,
 7620: 1,
 7696: 1,
 7832: 1,
 7848: 1,
 12474: 1,
 5093: 1,
 21949: 1,
 8489: 1,
 8535: 1,
 8684: 1,
 8732: 1,
 1324: 1,
 8934: 1,
 9073: 1,
 9256: 1,
 9364: 1,
 9617: 1,
 4110: 1,
 9779: 1,
 9921: 1,
 10118: 1,
 5411: 1,
 10210: 1,
 10270: 1,
 10340: 1,
 10538: 1,
 10889: 1,
 10902: 1,
 11011: 1,
 11176: 1,
 5605: 1,
 11468: 1,
 11598: 1,
 11603: 1,
 11624: 1,
 11707: 1,
 11837: 1,
 14609: 1,
 4815: 1,
 12027: 1,
 6279: 1,
 12255: 1,
 5782: 1,
 12537: 1,
 8568: 1,
 127

In [663]:
neighbors

{0: [18427],
 18427: [0,
  295,
  396,
  1099,
  1365,
  1898,
  17807,
  2257,
  2632,
  3179,
  3181,
  3897,
  4783,
  4809,
  4831,
  5371,
  5742,
  5934,
  6708,
  22322,
  8289,
  8358,
  8425,
  8762,
  8945,
  10101,
  19189,
  11475,
  11763,
  12040,
  6821,
  12726,
  13166,
  14266,
  14764,
  15356,
  15921,
  16687,
  16733,
  16979,
  17959,
  17963,
  18038,
  18165,
  18229,
  19382,
  20845,
  21808,
  21437,
  19550,
  20150],
 1: [21708,
  22208,
  22171,
  6829,
  16590,
  20135,
  8894,
  15785,
  10281,
  22265,
  7136,
  22405,
  10379,
  13737,
  8533,
  14344,
  2812,
  5755,
  16260,
  15026,
  17370,
  17460,
  8049,
  5307,
  4987,
  18304,
  12305,
  19743,
  20024,
  21729,
  10554,
  11557,
  5228,
  9934],
 21708: [1,
  123,
  130,
  343,
  356,
  375,
  426,
  9968,
  622,
  701,
  739,
  5241,
  5354,
  1041,
  1387,
  1557,
  1823,
  11671,
  9260,
  1946,
  2387,
  2596,
  2676,
  2812,
  3097,
  3448,
  3520,
  3707,
  20135,
  5228,
  3735,
  376

In [664]:
tot = sum(deg.values())/2
tot # e(S,S)

171002.0

In [665]:
ctr = len(deg) # |S|
ctr

22470

In [666]:
total_vertices = len(deg)

In [667]:
density = [] # stores densities
vertex = [] # vertex[0:i] (i included) contains vertices removed to achive density[i]

In [668]:
def find_density_greedy():
    global tot
    global deg
    global ctr
    global edges_df
    global vertex
    
    v = min(deg, key=lambda k: deg[k]) # vertex with minimum degree
    vertex.append(v) # Store the removed vertex
    ctr = ctr - 1 # decrease |S| by 1 (as v is removed)
    tot = tot - deg[v] # Update e(S,S)
    
    # Check for self edges, in that case, we subtracted 2, but we must subtract only one
    # selected_rows = edges_df[(edges_df['id_1'] == v) & (edges_df['id_2'] == v)]
    # tot += len(selected_rows)
    if v in self_loops:
        tot += self_loops[v]
    # if(len(selected_rows)!=0):
    #     print(selected_rows)

    den = tot / ctr
    density.append(den)
    # print(v)
    # deg[v] = 0
    deg.pop(v) # Remove v from deg

    selected_rows = pd.Series(neighbors[v])
    # print(selected_rows)
    deg_update = selected_rows.value_counts().to_dict()
    # print(deg_update)
    for ver, count in deg_update.items():
        if ver in deg:
            deg[ver] -= count

    # selected_rows = edges_df[edges_df['id_1'] == v]
    # # print(selected_rows)
    
    # deg_update = selected_rows['id_2'].value_counts().to_dict()
    # # print(deg_update)
    # for ver, count in deg_update.items():
    #     if ver in deg: # ver might be removed
    #         deg[ver] -= count

    # selected_rows = edges_df[edges_df['id_2'] == v]
    # # print(selected_rows)
    
    # deg_update = selected_rows['id_1'].value_counts().to_dict()
    # # print(deg_update)
    # for ver, count in deg_update.items():
    #     if ver in deg:
    #         deg[ver] -= count

    # edges_df = edges_df[~edges_df.isin([v]).any(axis=1)]
    
    
    



In [669]:
find_density_greedy()

In [670]:
tot

171001.0

In [671]:
sum(deg.values())

342002

In [672]:
density

[7.610530063643242]

In [673]:
i = 1
while(ctr>1):
    i+=1
    if (i%1000==0):
        print(i)
    find_density_greedy()
    if(tot != sum(deg.values())/2):
        print("here")
        # break

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000


In [674]:
tot

0.0

In [675]:
sum(deg.values())/2

0.0

In [676]:
density = np.array(density)

In [677]:
np.argmax(density)

22148

In [678]:
x = np.argmax(density)

In [679]:
density[x]

35.006230529595015

In [680]:
graph = []
for i in range(0, total_vertices):
    if i not in vertex[:x+1]:
        graph.append(i)
    

In [681]:
len(graph)

321

In [682]:
fullgraph = {}
for ver in range(total_vertices):
    type = df.loc[ver, 'page_type']
    if(type in fullgraph.keys()):
        fullgraph[type] += 1
    else:
        fullgraph[type] = 1

fullgraph

{'tvshow': 3327, 'government': 6880, 'company': 6495, 'politician': 5768}

In [683]:
dense1 = {}
for ver in graph:
    type = df.loc[ver, 'page_type']
    if(type in dense1.keys()):
        dense1[type] += 1
    else:
        dense1[type] = 1

dense1

{'tvshow': 60, 'company': 4, 'government': 257}

In [684]:
density[np.argmin(density)]

0.0

In [685]:
deg

{7606: 0}

In [686]:
df

,id,facebook_id,page_name,page_type
0,0,145647315578475,The Voice of China 中国好声音,tvshow
1,1,191483281412,U.S. Consulate General Mumbai,government
2,2,144761358898518,ESET,company
3,3,568700043198473,Consulate General of Switzerland in Montreal,government
4,4,1408935539376139,Mark Bailey MP - Labor for Miller,politician
...,...,...,...,...
22465,22465,1379955382222841,Kurt Wiegel MdL,politician
22466,22466,1651527995097082,dubdub Stories,company
22467,22467,155369444540412,Ministerio del Interior - Paraguay,government
22468,22468,175067819212798,Tottus Perú,company


In [687]:
len(graph)

321

In [688]:
df[df['id'].isin(graph)]


,id,facebook_id,page_name,page_type
44,44,1507698529534072,APB FOX,tvshow
127,127,354595427910013,GovX,company
159,159,167394579972573,JRTC and Fort Polk,government
568,568,127428534470290,The Orville,tvshow
679,679,146451038734267,Michigan National Guard,government
...,...,...,...,...
22252,22252,174788155866586,U.S. Army School of Music,government
22265,22265,61575637587,U.S. Pacific Command,government
22328,22328,114802901909136,North Carolina National Guard,government
22349,22349,110318209007293,21st Theater Sustainment Command,government
